# w261 Final Project - Clickthrough Rate Prediction


[Your team number (from the spreadsheet)]   
[Your team names]   
Summer 2019, section [Your section numbers>]   

## Table of Contents

* __Section 1__ - Question Formulation
* __Section 2__ - Algorithm Explanation
* __Section 3__ - EDA & Challenges
* __Section 4__ - Algorithm Implementation
* __Section 5__ - Course Concepts

# __Section 1__ - Question Formulation

# __Section 2__ - Algorithm Explanation

# __Section 3__ - EDA & Challenges

# __Section 4__ - Algorithm Implementation

# __Section 5__ - Course Concepts

### Setup and Initiate Spark

In [1]:
import re
import ast
import time
import itertools
import numpy as np
from numpy import allclose
import pandas as pd
import matplotlib.pyplot as plt
from pyspark.sql import Row
from pyspark.sql import SQLContext
import pyspark.sql.functions as F
from pyspark.sql.functions import *
from pyspark.ml.classification import  RandomForestClassifier
from pyspark.ml.feature import StringIndexer, OneHotEncoderEstimator, VectorAssembler, VectorSlicer
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.linalg import Vectors
from pyspark.mllib.linalg import SparseVector
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit

In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
# store path to notebook
PWD = !pwd
PWD = PWD[0]

In [4]:
# start Spark Session (RUN THIS CELL AS IS)
from pyspark.sql import SparkSession
app_name = "hw3_notebook"
master = "local[*]"
spark = SparkSession\
        .builder\
        .appName(app_name)\
        .master(master)\
        .getOrCreate()
sc = spark.sparkContext
sqlContext = SQLContext(sc)

### Setup Path and read in raw data files

In [7]:
# Read the toy data file
toy_raw = ast.literal_eval(open("data/toy.txt", "r").read())

# Read the 10K sample data file for feature extraction and data preparation
tenK_raw = ast.literal_eval(open("data/eda.txt", "r").read())

### Parse Row Into Readable formats

In [8]:
def parse_raw_row(row):
    '''
    for each row in the raw data,  output is a list of label and all the features:
        - [label, feature_1, feature_1, ...]
    For first 13 features, change the data type to number.
    Remaining features will of type string.
    For null values, populate None
    '''
    row_values = row.split('\t')
    for i, value in enumerate(row_values):
        if i <14:
            row_values[i] = int(value) if value != '' else None
        else:
            row_values[i] = value if value != '' else "''"
    return row_values

In [10]:
# parse raw toy data to form toyRDD
toyRDD = sc.parallelize(toy_raw).map(parse_raw_row).cache()
#toyRDD.take(1)

# parse raw 10k sample data to form tenKRDD
tenKRDD = sc.parallelize(tenK_raw).map(parse_raw_row).cache()

## Create Dataframe for Feature Engineering ##

In [11]:
#### Create SQL dataframe from RDD

# for toy data
toyFeature_df = sqlContext.createDataFrame(toyRDD)

# for 10K sample data
tenKfeature_df = sqlContext.createDataFrame(tenKRDD)

## Feature Extraction

**1: Remove features with very large number of unknown data**  

From EDA we see that the below feature columns have more than 40% null values.  
Due to this high percenage of unknown data we won't keep these features in our model. So dropping these columns from our data frame.

In [12]:
# drop features with high unknown values

toy_df1 = toyFeature_df.drop('_13','_36','_2','_11','_33','_34','_39','_40')
tenK_df1 = tenKfeature_df.drop('_13','_36','_2','_11','_33','_34','_39','_40')

tenK_df1.show(1)

+---+---+---+---+---+---+---+---+---+---+---+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
| _1| _3| _4| _5| _6| _7| _8| _9|_10|_12|_14|     _15|     _16|     _17|     _18|     _19|     _20|     _21|     _22|     _23|     _24|     _25|     _26|     _27|     _28|     _29|     _30|     _31|     _32|     _35|     _37|     _38|
+---+---+---+---+---+---+---+---+---+---+---+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
|  0|  6|  3| 34|  4| 27|  3| 33| 34|  1| 15|05db9164|028bd518|77f2f2e5|d16679b9|25c83c98|fbad5c96|2ecb612f|5b392875|a73ee510|3b08e48b|4efc1873|9f32b866|f15f3681|b28479f6|9559bea6|31ca40b6|07c540c4|2a40f0da|dfcfc3fa|32c7478e|aee52b6f|
+---+---+---+---+---+---+---+---+---+---+---+--------+------

**2. Remove Categorical features with high % of Uniqueness of Categories**  

From EDA, we see that for the following categorical features uniqueness is more than 50%. When uniqueness of a feature is more than 50% it should not be having much impact on label prediction. So will remove those columns from our model.  
_17, _18, _21, _24, _26, _30, _35

In [13]:
toy_df2 = toy_df1.drop('_17','_18','_21','_24','_26','_30','_35')
tenK_df2 = tenK_df1.drop('_17','_18','_21','_24','_26','_30','_35')
tenK_df2.show(1)

+---+---+---+---+---+---+---+---+---+---+---+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
| _1| _3| _4| _5| _6| _7| _8| _9|_10|_12|_14|     _15|     _16|     _19|     _20|     _22|     _23|     _25|     _27|     _28|     _29|     _31|     _32|     _37|     _38|
+---+---+---+---+---+---+---+---+---+---+---+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
|  0|  6|  3| 34|  4| 27|  3| 33| 34|  1| 15|05db9164|028bd518|25c83c98|fbad5c96|5b392875|a73ee510|4efc1873|f15f3681|b28479f6|9559bea6|07c540c4|2a40f0da|32c7478e|aee52b6f|
+---+---+---+---+---+---+---+---+---+---+---+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
only showing top 1 row



**3. Colenearity Reduction**  

EDA shows strong correlation between below numerical features:  
> _5 and _14  
> _5 and _9  
> _9 and _14  
> _8 and _12  
There is also a moderate negative correlation for feature:  
> _6 and _11  
> _7 and _11.  
To avoid co-leniarity we will remove feature _14, _9, _6, _7 and _8

In [14]:
toy_df3 = toy_df2.drop('_17','_18','_21','_24','_26','_30','_35')
tenK_df3 = tenK_df2.drop('_17','_18','_21','_24','_26','_30','_35')
tenK_df3.show(1)

+---+---+---+---+---+---+---+---+---+---+---+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
| _1| _3| _4| _5| _6| _7| _8| _9|_10|_12|_14|     _15|     _16|     _19|     _20|     _22|     _23|     _25|     _27|     _28|     _29|     _31|     _32|     _37|     _38|
+---+---+---+---+---+---+---+---+---+---+---+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
|  0|  6|  3| 34|  4| 27|  3| 33| 34|  1| 15|05db9164|028bd518|25c83c98|fbad5c96|5b392875|a73ee510|4efc1873|f15f3681|b28479f6|9559bea6|07c540c4|2a40f0da|32c7478e|aee52b6f|
+---+---+---+---+---+---+---+---+---+---+---+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
only showing top 1 row



**4. Replace null values in numerical variables with mean**

In [15]:
def fill_with_mean(this_df):

    '''
    replace None value in Numerical variables with mean value
    '''
    stats = this_df.agg(*(avg(c).alias(c) for i,c in enumerate(this_df.columns) if i<11 and c != '_1'))
    return this_df.na.fill(stats.first().asDict())


In [17]:
tenK_df4 = fill_with_mean(tenK_df3)
tenK_df4.cache()
tenK_df4.show(1,False)

+---+---+---+---+---+---+---+---+---+---+---+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
|_1 |_3 |_4 |_5 |_6 |_7 |_8 |_9 |_10|_12|_14|_15     |_16     |_19     |_20     |_22     |_23     |_25     |_27     |_28     |_29     |_31     |_32     |_37     |_38     |
+---+---+---+---+---+---+---+---+---+---+---+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
|0  |6  |3  |34 |4  |27 |3  |33 |34 |1  |15 |05db9164|028bd518|25c83c98|fbad5c96|5b392875|a73ee510|4efc1873|f15f3681|b28479f6|9559bea6|07c540c4|2a40f0da|32c7478e|aee52b6f|
+---+---+---+---+---+---+---+---+---+---+---+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
only showing top 1 row



**5: Binning of Categories**  

From EDA we see that amongst remaining categorical features, unique category count for following  are very low.  
_23, _20    
For rest of the features we will try binning the categories into three distinct features. At the same time converting the feature type to numerical.  
Binning is done by checking click through frequency ratio: (count of 1/count of totalfrequency)  
If frequency ratio > 0.5, value = "A"  
0.5> frequency ratio > 0.2, value = "B"  
Otherwise value = "C"

For null values populating one of the bin value-s ("A", "B", "C") randomly

> Check frequency of unique categories of Categorical Features

In [18]:
# Calculate click through rate frequency count of each category


exclude_list = ['_23', '_20'] #, '_31'
 
tenK_click_df = tenK_df4
for n,i in enumerate(tenK_df4.dtypes):

    if i[1]=='string' and i[0] not in exclude_list:
        
        feature = i[0]

        # frequency count of unique categories under each feature
        cat_freqDF = tenK_df4.groupBy(feature).count()
        
        # click through frequency count: count of label = 1 for each category
        click_freqDF = tenK_df4.where("_1 == 1").groupBy(feature, "_1").count()
        
        
        ## Calculate click through frequency ratio for each category:
        ##(count of label = 1/total count)
        
        df1 = click_freqDF.alias('df1')
        df2 = cat_freqDF.alias('df2')
        if n == 0:
            df3 = tenK_df4.alias('df3')
        else:
            df3 = tenK_click_df.alias('df3')

        tenK_click_df = df1.join(df2, [feature]).join(df3, [feature]).select(feature, 'df3.*',
                                (df1['count']/df2['count']).alias(feature+"_click"))

        ## End of click through frequency ratio calculation       
        
        ###### Bin data into three categories based on the frequency percentage count.
        # if frequency % of a category is greather/ equal to 50%, assigned value =30
        # if frequency % of a category is greater than 19.99% but less than 50%, value = 20
        # for the rest value is 10
        tenK_click_df = tenK_click_df.withColumn(feature,
        F.when(((tenK_click_df[feature+'_click'] > 0.5)|(tenK_click_df[feature+'_click'] == 0.5))
           & (tenK_click_df[feature] != ''),F.lit("A"))
        .otherwise(
        F.when((tenK_click_df[feature+'_click'] < 0.5) & (tenK_click_df[feature+'_click'] > 0.2)
           & (tenK_click_df[feature] != ''),F.lit("B"))
        .otherwise(
        F.when(tenK_click_df[feature] == '', F.array(F.lit("C"),F.lit("B"),F.lit("A")).getItem((F.rand()*3).cast("int")))
        .otherwise(F.lit("C")))))
                
   
        
    elif i[0] == '_23':
        feature = '_23'
        
        if n == 0:
            temp_df = tenK_df4
        else:
            temp_df = tenK_click_df

        tenK_click_df = temp_df.withColumn(feature,
        F.when(temp_df[feature] == 'a73ee510', F.lit("A"))
        .otherwise(
        F.when(temp_df[feature] == '7cc72ec2', F.lit("B"))
        .otherwise(
        F.when(temp_df[feature] == '', F.array(F.lit("C"),F.lit("B"),F.lit("A")).getItem((F.rand()*3).cast("int")))
        .otherwise(F.lit("C")))))
            
    elif i[0] == '_20':
        feature = '_20'    
        
        if n == 0:
            temp_df = tenK_df4
        else:
            temp_df = tenK_click_df
            
        tenK_click_df = temp_df.withColumn(feature,
        F.when(temp_df[feature] == '7e0ccccf', F.lit("A"))
        .otherwise(
        F.when(temp_df[feature] == ('fbad5c96'), F.lit("B"))
        .otherwise(
        F.when(temp_df[feature] == ('fe6b92e5'), F.lit("C"))
        .otherwise(
        F.when(temp_df[feature] == '',
                F.array(F.lit("C"),F.lit("B"),F.lit("A"),F.lit("D")).getItem((F.rand()*4).cast("int")))
        .otherwise(F.lit("D"))))))

tenK_df5 = tenK_click_df.drop('_15_click','_16_click','_19_click','_22_click','_25_click','_27_click',
                             '_28_click','_29_click', '_31_click', '_32_click', '_37_click', '_38_click')
tenK_df5.cache()

DataFrame[_38: string, _37: string, _32: string, _31: string, _29: string, _28: string, _27: string, _25: string, _22: string, _19: string, _16: string, _15: string, _1: bigint, _3: bigint, _4: bigint, _5: bigint, _6: bigint, _7: bigint, _8: bigint, _9: bigint, _10: bigint, _12: bigint, _14: bigint, _20: string, _23: string]

In [19]:
tenK_df5.show(20,False)

+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+------+---+---+---+----+---+---+---+---+
|_38|_37|_32|_31|_29|_28|_27|_25|_22|_19|_16|_15|_1 |_3 |_4 |_5 |_6    |_7 |_8 |_9 |_10 |_12|_14|_20|_23|
+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+------+---+---+---+----+---+---+---+---+
|A  |B  |B  |B  |B  |B  |C  |C  |B  |B  |C  |B  |1  |0  |2  |2  |13140 |638|7  |20 |600 |4  |8  |B  |A  |
|A  |B  |A  |B  |A  |B  |B  |B  |B  |B  |B  |B  |1  |3  |1  |1  |6396  |11 |15 |1  |147 |7  |1  |B  |A  |
|A  |C  |B  |C  |C  |B  |A  |A  |B  |B  |B  |B  |1  |79 |37 |7  |678   |111|0  |15 |15  |0  |8  |A  |A  |
|B  |C  |B  |B  |B  |B  |B  |B  |B  |B  |B  |A  |1  |30 |33 |7  |3720  |4  |98 |0  |86  |10 |8  |A  |A  |
|B  |B  |B  |B  |B  |B  |B  |A  |B  |B  |B  |B  |0  |9  |2  |10 |1248  |19 |2  |14 |16  |1  |13 |C  |A  |
|B  |B  |B  |B  |B  |B  |B  |B  |B  |B  |B  |B  |0  |10 |60 |20 |0     |177|48 |29 |191 |9  |0  |D  |A  |
|B  |B  |B  |B  |B  |B  |A  |A  |B  |B  |B  |B

**6: Run RandomForest ensemble to check featureImportances**:  
With remaining features we will run RandomForest ensemble classifier to check featureImportances matrices.  
We will extract the Features with higher featureImportances scores for our model.  


In [20]:
# One hot encoding and assembling using Random Forest Classifier

encoding_var = [i[0] for i in tenK_df5.dtypes if (i[1]=='string')]
num_var = [i[0] for i in tenK_df5.dtypes if (i[1]!='string') & (i[0]!= '_1')]

string_indexes = [StringIndexer(inputCol = c, outputCol = 'IDX_' + c, handleInvalid = 'keep')
                  for c in encoding_var]
onehot_indexes = [OneHotEncoderEstimator(inputCols = ['IDX_' + c], outputCols = ['OHE_' + c])
                  for c in encoding_var]
label_indexes = StringIndexer(inputCol = '_1', outputCol = 'label', handleInvalid = 'keep')
assembler = VectorAssembler(inputCols = num_var + ['OHE_' + c for c in encoding_var]
                            , outputCol = "features")
rf = RandomForestClassifier(labelCol="label", featuresCol="features", seed = 8464,
                             numTrees=10, cacheNodeIds = True, subsamplingRate = 0.7)

pipe = Pipeline(stages = string_indexes + onehot_indexes + [assembler, label_indexes, rf])

In [21]:
## fit into pipe

mod = pipe.fit(tenK_df5)
tenK_df6 = mod.transform(tenK_df5)
#mod.stages[-1].featureImportances

In [22]:
def ExtractFeatureImp(featureImp, dataset, featuresCol):
    '''
    Function to display featureImportances in human readable format
    '''
    list_extract = []
    for i in dataset.schema[featuresCol].metadata["ml_attr"]["attrs"]:
        list_extract = list_extract + dataset.schema[featuresCol].metadata["ml_attr"]["attrs"][i]
    varlist = pd.DataFrame(list_extract)
    varlist['score'] = varlist['idx'].apply(lambda x: featureImp[x])
    return(varlist.sort_values('score', ascending = False))

In [23]:
### Print featureImportance of each feature
ExtractFeatureImp(mod.stages[-1].featureImportances, tenK_df6, "features")

,idx,name,score
12,12,OHE__38_A,0.228731
4,4,_7,0.135129
23,23,OHE__29_A,0.118916
32,32,OHE__25_A,0.094095
29,29,OHE__27_A,0.087833
22,22,OHE__29_C,0.050605
11,11,OHE__38_C,0.046733
31,31,OHE__25_C,0.037637
18,18,OHE__32_A,0.037563
3,3,_6,0.033223
